<a href="https://colab.research.google.com/github/MastersMasterM/DataDrivenModelingProject/blob/main/JANOS_DataDrivenModeling_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing Project Setting

## Installing Packages

In [ ]:
!pip install pyomo

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
import warnings
# Suppress the FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

## Importing Datasets

In [ ]:
!pip install gdown

In [ ]:
#Downloading Dataset
!gdown 1WM1zfkG7n9_QLdhTFQwRWu1Ky35GvcCA
!gdown 1HjYbKpxsyY03K3K1-vi9A9mcIeNnYwp_

Downloading...
From: https://drive.google.com/uc?id=1WM1zfkG7n9_QLdhTFQwRWu1Ky35GvcCA
To: /content/college_applications6000.csv
100% 94.2k/94.2k [00:00<00:00, 30.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HjYbKpxsyY03K3K1-vi9A9mcIeNnYwp_
To: /content/college_student_enroll-s1-1.csv
100% 568k/568k [00:00<00:00, 23.5MB/s]


In [ ]:
import pandas as pd
# Read the dataset from a file (assuming a CSV file)
trainset = pd.read_csv('/content/college_student_enroll-s1-1.csv')
testset = pd.read_csv('/content/college_applications6000.csv')

## Standardizing Datasets

In [ ]:
from sklearn.preprocessing import StandardScaler
#Scaling Testset
SAT = StandardScaler().fit(testset[['SAT']])
GPA = StandardScaler().fit(testset[['GPA']])

testset['SAT'] = SAT.transform(testset[["SAT"]])
testset['GPA'] = GPA.transform(testset[["GPA"]])

In [ ]:
# Normalizing Trainingset
SAT = StandardScaler().fit(trainset[['SAT']])
GPA = StandardScaler().fit(trainset[['GPA']])
trainset['SAT'] = SAT.transform(trainset[["SAT"]])
trainset['GPA'] = GPA.transform(trainset[["GPA"]])

In [ ]:
trainset.shape

(20000, 6)

In [ ]:
testset.shape

(6000, 3)

In [ ]:
trainset.enroll.value_counts()

0    10000
1    10000
Name: enroll, dtype: int64

## Tuning MLP

In [ ]:
# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

In [ ]:
from sklearn.preprocessing import StandardScaler
SAT = StandardScaler().fit(X[['SAT']])
GPA = StandardScaler().fit(X[['GPA']])
X['SAT'] = SAT.transform(X[["SAT"]])
X['GPA'] = GPA.transform(X[["GPA"]])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'beta_1': [0.9, 0.95, 0.99],
    'beta_2': [0.999, 0.9999],
    'epsilon': [1e-8, 1e-7, 1e-6],
}

In [ ]:
%%time
from sklearn.neural_network import MLPRegressor

dict_l_alpha = {}
for l in range(1,4):
  #Setting Nurons number for each layer
  n_size = [10] * l
  n_size = tuple(n_size)

  # Create an MLP classifier with fixed activation and hidden layer size
  mlp = MLPRegressor(hidden_layer_sizes=n_size, random_state=666, max_iter=500, solver='adam', activation = 'relu')

  # Create a GridSearchCV object and fit it to the training data
  grid_search = GridSearchCV(mlp, param_grid, cv=5)
  grid_search.fit(X_train, y_train)

  # Get the best hyperparameters and the corresponding model
  best_params = grid_search.best_params_
  best_model = grid_search.best_estimator_

  # Evaluate the best model on the test data
  accuracy = best_model.score(X_test, y_test)

  # Print the best hyperparameters and the accuracy
  print(f"Best Hyperparameters for l = {l}: {best_params}")
  print(f"Accuracy for l = {l}: {accuracy}")
  dict_l_alpha[l] = best_params

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Saved Result From Tuning

In [ ]:
dict_l_alpha = {}

dict_l_alpha[1] = {'beta_1': 0.95, 'beta_2': 0.999, 'epsilon': 1e-08}
dict_l_alpha[2] = {'beta_1': 0.95, 'beta_2': 0.999, 'epsilon': 1e-07}
dict_l_alpha[3] = {'beta_1': 0.95, 'beta_2': 0.999, 'epsilon': 1e-06}

# Implemented JANOS

## Modeling Phase

In [ ]:
import pyomo.environ as pyo
import numpy as np

def Modeling(N:int, l:int, testset:pd.DataFrame, weights:np.array, bias:np.array, inp_shape:int, n_neurons:int=10, discrete:bool=False):

  #Getting a sample of size N from test data
  testset = testset.sample(N+1)
  testset = testset.reset_index(drop=True)

  #Computing Number of Neurons
  n_size = [n_neurons] * l
  n_size = tuple(n_size)

  #Create Concerete Model Pyo Model
  model = pyo.ConcreteModel()

  #Constants
  model.eps = 1e-08
  model.M = 5000.0

  #Parameters
  model.N = N
  model.l = l
  model.n_input_layer = inp_shape
  model.n_output_layer = 1
  model.n_hidden_neurons = l*10
  model.n_neurons = inp_shape + l*10 + 1

  #Sets
  model.i = pyo.RangeSet(1, model.N)
  model.neurons = pyo.RangeSet(1, model.n_neurons)
  model.input_layer = pyo.RangeSet(1, model.n_input_layer)
  model.hidden_neurons = pyo.RangeSet(1 + model.n_input_layer, model.n_input_layer + model.n_hidden_neurons)
  model.output_layer = pyo.RangeSet(1 + model.n_input_layer + model.n_hidden_neurons, model.n_input_layer + model.n_hidden_neurons + 1)
  model.layer_except_inp = model.hidden_neurons | model.output_layer
  model.D = pyo.Set(initialize=[0, 0.5, 1, 1.5, 2, 2.5])

  #Variables
  model.G = pyo.Var(model.i, model.neurons, domain=pyo.Reals)
  model.F = pyo.Var(model.i, model.neurons, domain=pyo.Reals)
  model.z = pyo.Var(model.i, model.neurons, domain=pyo.Binary)
  model.y = pyo.Var(model.i, domain=pyo.Reals)
  if discrete:
    model.x = pyo.Var(model.i, domain=model.D)
  else:
    model.x = pyo.Var(model.i, domain=pyo.Reals)

  #Parameters

  #Weights
  model.W = pyo.Param(model.neurons, model.neurons, initialize=0.0, mutable=True)
  # Assigning values to model.W based on MLP weights layer by layer
  for i in model.input_layer:
    for j in model.hidden_neurons:
      if j <= (model.n_input_layer + 10):
        model.W[i, j] = round(weights[0][i - 1][j - model.n_input_layer - 1], 6)

  # Hidden layers
  for layer in range(1, model.l):
    # Connect neurons in the current hidden layer to neurons in the previous hidden layer or input layer
    for i in range(model.n_input_layer + (layer-1) * 10 + 1, model.n_input_layer + (layer) * 10 + 1):
      for j in range(model.n_input_layer + (layer) * 10 + 1, model.n_input_layer + (layer + 1) * 10 + 1):
        model.W[i, j] = round(weights[layer][i - (model.n_input_layer + (layer-1) * 10) - 1][j - (model.n_input_layer + (layer) * 10) - 1], 6)

  layer = model.l

  # Last hidden layer to output layer
  for i in range(model.n_input_layer + (layer-1) * 10 + 1, model.n_input_layer + (layer) * 10 + 1):
    j = model.n_input_layer + (layer) * 10 + 1
    model.W[i, j] = round(weights[layer][i - (model.n_input_layer + (layer-1) * 10) - 1][0], 6)

  #Bias
  model.b = pyo.Param(model.neurons, initialize=0.0, mutable=True)

  # Hidden Layer Neurons Bias
  for i in range(model.l):
    layer_bias = bias[i]
    for j in range(model.n_input_layer + (i) * 10 + 1, model.n_input_layer + (i+1) * 10 + 1):
      model.b[j] = round(layer_bias[j-(model.n_input_layer + (i) * 10 + 1)], 6)

  # Last hidden layer to output layer
  for j in model.output_layer:
      model.b[j] = round(bias[-1][0], 6)
  #Constraints

  #Budget Constraint
  model.Constraint1 = pyo.Constraint(expr = sum(1 * model.x[i] for i in model.i) <= int(0.2*N))

  #Regular Variable Constraint
  def x_constraint_rule(m, i):
      # return the expression for the constraint for i
      return m.x[i] <= 2.5
  model.xbConstraint = pyo.Constraint(model.i, rule=x_constraint_rule)

  def x_constraint_rule2(m, i):
      # return the expression for the constraint for i
      return 0.0 <= m.x[i]
  model.xbConstraint2 = pyo.Constraint(model.i, rule=x_constraint_rule2)

  #Predictive

  #Input Layer
  def g1_constraint_rule(m, i):
    sat = testset.loc[i, 'SAT']
    return m.G[i, 1] == sat
  model.g1Constraint = pyo.Constraint(model.i, rule=g1_constraint_rule)

  def g2_constraint_rule(m, i):
    gpa = testset.loc[i, 'GPA']
    return m.G[i, 2] == gpa
  model.g2Constraint = pyo.Constraint(model.i, rule=g2_constraint_rule)

  def g3_constraint_rule(m, i):
    return m.G[i, 3] == m.x[i]
  model.g3Constraint = pyo.Constraint(model.i, rule=g3_constraint_rule)

  #Input and Output Constraints
  def f1_constraint_rule(m, i):
    return m.G[i, 1] == m.F[i, 1]
  model.f1Constraint = pyo.Constraint(model.i, rule=f1_constraint_rule)

  def f2_constraint_rule(m, i):
    return m.G[i, 2] == m.F[i, 2]
  model.f2Constraint = pyo.Constraint(model.i, rule=f2_constraint_rule)

  def f3_constraint_rule(m, i):
    return m.G[i, 3] == m.F[i, 3]
  model.f3Constraint = pyo.Constraint(model.i, rule=f3_constraint_rule)

  def ft_constraint_rule(m, i):
    return m.G[i, m.n_neurons] == m.F[i, m.n_neurons]
  model.ftConstraint = pyo.Constraint(model.i, rule=ft_constraint_rule)

  #Hidden Layers
  def hidden_layers_rule(m, i, j):
    if j <= m.n_input_layer + 10:
      summation = sum(m.W[k, j] * m.F[i, k] for k in m.input_layer)
      summation += m.b[j]
      return summation == m.G[i, j]
    else:
      layer_num = ((j - m.n_input_layer - 1) // 10) + 1
      previous_layer = range(m.n_input_layer + (layer_num - 2) * 10 + 1, m.n_input_layer + (layer_num - 1) * 10 + 1)
      summation = sum(m.W[k, j] * m.F[i, k] for k in previous_layer)
      summation += m.b[j]
      return summation == m.G[i, j]
  model.hidden_layerConstraint = pyo.Constraint(model.i, model.layer_except_inp, rule=hidden_layers_rule)

  #Binary Rules
  def binary_rule1(m, i, j):
    return m.G[i, j] - m.M * m.z[i, j] <= 0
  model.binary1Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule1)

  def binary_rule2(m, i, j):
    return m.G[i, j] + m.M * (1 - m.z[i, j]) >= m.eps
  model.binary2Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule2)

  def binary_rule3(m, i, j):
    return m.G[i, j] + m.M * (1 - m.z[i, j]) - m.F[i, j] + m.eps >= 0
  model.binary3Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule3)

  def binary_rule4(m, i, j):
    return m.G[i, j] - m.M * (1 - m.z[i, j]) - m.F[i, j] <= m.eps
  model.binary4Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule4)

  def binary_rule5(m, i, j):
    return m.F[i, j] >= 0
  model.binary5Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule5)

  def binary_rule6(m, i, j):
    return m.F[i, j] <= m.z[i, j] * m.M + m.eps
  model.binary6Constraint = pyo.Constraint(model.i, model.hidden_neurons, rule=binary_rule6)

  #Setting Predicted Variable
  def y_constraint_rule(m, i):
    return m.y[i] == m.F[i, m.n_neurons]

  model.ybConstraint = pyo.Constraint(model.i, rule=y_constraint_rule)

  #Objective Function
  def obj_rule(m):
      return sum(m.y[i] for i in m.i)

  model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

  return model, testset

## Running Implemented Version



### Different MLP Solvers

#### Adam

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#Setting Nurons number per hidden layers
n_neurons = 10
l = 1
n_size = l * [n_neurons]

# Create an MLPRegressor object
mlp = MLPRegressor(hidden_layer_sizes=n_size, activation='relu', max_iter=500, solver='adam', random_state=3, **dict_l_alpha[l])

# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Fit the MLP to the data to compute the initial weights
mlp.fit(X_train, y_train)

#Getting weights and bias
weights = mlp.coefs_
bias = mlp.intercepts_

In [ ]:
mlp.score(X_test, y_test)

0.9612498114793685

In [ ]:
%%time
import pandas as pd
import time
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor

N = 25
l = 1
d = False

model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

start_time = time.time()
result = solver.solve(model)
solve_time = (time.time() - start_time)
print(f"Solve Time in Min: {solve_time/60}")
pyo.display(model.obj)

Solve Time in Min: 0.09361316363016764
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 14.238627979819999
CPU times: user 211 ms, sys: 77.2 ms, total: 288 ms
Wall time: 5.68 s


In [ ]:
model.y.pprint()

y : Size=25, Index=i
    Key : Lower : Value          : Upper : Fixed : Stale : Domain
      1 :  None :      1.0011246 :  None : False : False :  Reals
      2 :  None :      1.0043806 :  None : False : False :  Reals
      3 :  None :     0.92351533 :  None : False : False :  Reals
      4 :  None :       1.008626 :  None : False : False :  Reals
      5 :  None :     0.96961991 :  None : False : False :  Reals
      6 :  None :  -0.0019753724 :  None : False : False :  Reals
      7 :  None :   0.0051982731 :  None : False : False :  Reals
      8 :  None :     0.95577287 :  None : False : False :  Reals
      9 :  None : -0.00070213698 :  None : False : False :  Reals
     10 :  None :   0.0068685858 :  None : False : False :  Reals
     11 :  None :      1.0019228 :  None : False : False :  Reals
     12 :  None :      1.0009333 :  None : False : False :  Reals
     13 :  None :     0.99874105 :  None : False : False :  Reals
     14 :  None :     0.87026014 :  None : False : Fals

In [ ]:
model.x.pprint()

x : Size=25, Index=i
    Key : Lower : Value      : Upper : Fixed : Stale : Domain
      1 :  None :        0.0 :  None : False : False :  Reals
      2 :  None :        0.0 :  None : False : False :  Reals
      3 :  None : 0.43435651 :  None : False : False :  Reals
      4 :  None :        0.0 :  None : False : False :  Reals
      5 :  None :  1.2031416 :  None : False : False :  Reals
      6 :  None :        0.0 :  None : False : False :  Reals
      7 :  None :        0.0 :  None : False : False :  Reals
      8 :  None :   1.233705 :  None : False : False :  Reals
      9 :  None :        0.0 :  None : False : False :  Reals
     10 :  None :        0.0 :  None : False : False :  Reals
     11 :  None :        0.0 :  None : False : False :  Reals
     12 :  None :        0.0 :  None : False : False :  Reals
     13 :  None :        0.0 :  None : False : False :  Reals
     14 :  None : 0.49794974 :  None : False : False :  Reals
     15 :  None :        0.0 :  None : False : Fa

##### Computing Average Performance

In [ ]:
N = 25
l = 1
d = False

res_adam = []
while len(res_adam) <= 30:
  model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
  solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

  start_time = time.time()
  result = solver.solve(model)
  solve_time = (time.time() - start_time)
  print(f"Solve Time in Min: {solve_time/60}")
  # check the status and termination condition of the result
  if result.solver.status == pyo.SolverStatus.ok and result.solver.termination_condition == pyo.TerminationCondition.optimal:
    # access the objective value
    obj_val = pyo.value(model.obj)
    print(f"Objective value: {obj_val}")
    # save the objective value to the res list
    res_adam.append(obj_val)
  else:
    # the problem was not solved optimally
    print("The problem was not solved optimally")
    print(f"Termination condition: {result.solver.termination_condition}")

Solve Time in Min: 0.05118072032928467
Objective value: 13.63252032426
Solve Time in Min: 0.0673355499903361
Objective value: 14.253937030020003
Solve Time in Min: 0.026890607674916585
Objective value: 12.869720185299999
Solve Time in Min: 0.1227588136990865
Objective value: 15.662252798249998
Solve Time in Min: 0.03530945380528768
Objective value: 19.748395273610004
Solve Time in Min: 0.02674362262090047
Objective value: 13.779510951060002
Solve Time in Min: 0.008962007363637288
Objective value: 16.688803393729998
Solve Time in Min: 0.02846705913543701
Objective value: 14.840089351800003
Solve Time in Min: 0.04347480932871501
Objective value: 14.0549804929
Solve Time in Min: 0.022524627049763997
Objective value: 17.9380263593
Solve Time in Min: 0.059160431226094566
Objective value: 14.232068036288
Solve Time in Min: 0.013463421662648519
Objective value: 10.0065408499
Solve Time in Min: 0.012687929471333821
Objective value: 18.969754505999994
Solve Time in Min: 0.03710731267929077
Obje

In [ ]:
print(f"mean result = {np.mean(res_adam)}, std result = {np.std(res_adam)}")

mean result = 14.85695621312439, std result = 2.487781772895918


#### Adam (Not Tuned)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#Setting Nurons number per hidden layers
n_neurons = 10
l = 1
n_size = l * [n_neurons]

# Create an MLPRegressor object
mlp = MLPRegressor(hidden_layer_sizes=n_size)

# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Fit the MLP to the data to compute the initial weights
mlp.fit(X_train, y_train)

#Getting weights and bias
weights = mlp.coefs_
bias = mlp.intercepts_

In [ ]:
mlp.score(X_test, y_test)

0.9654042805107322

In [ ]:
%%time
import pandas as pd
import time
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor

N = 25
l = 1
d = False

model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

start_time = time.time()
result = solver.solve(model)
solve_time = (time.time() - start_time)
print(f"Solve Time in Min: {solve_time/60}")
pyo.display(model.obj)

Solve Time in Min: 0.019599302609761556
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 16.9500203361
CPU times: user 164 ms, sys: 91.7 ms, total: 255 ms
Wall time: 1.23 s


In [ ]:
model.y.pprint()

y : Size=25, Index=i
    Key : Lower : Value         : Upper : Fixed : Stale : Domain
      1 :  None :     1.0029565 :  None : False : False :  Reals
      2 :  None :  -0.005777494 :  None : False : False :  Reals
      3 :  None :  0.0028332092 :  None : False : False :  Reals
      4 :  None :     1.0042065 :  None : False : False :  Reals
      5 :  None : -0.0060023142 :  None : False : False :  Reals
      6 :  None :    0.98521295 :  None : False : False :  Reals
      7 :  None :    0.99172234 :  None : False : False :  Reals
      8 :  None :     1.0045025 :  None : False : False :  Reals
      9 :  None :     0.9771181 :  None : False : False :  Reals
     10 :  None :     1.0080585 :  None : False : False :  Reals
     11 :  None :    0.98841122 :  None : False : False :  Reals
     12 :  None :     1.0283158 :  None : False : False :  Reals
     13 :  None : -0.0029779348 :  None : False : False :  Reals
     14 :  None :      1.002398 :  None : False : False :  Reals
    

In [ ]:
model.x.pprint()

x : Size=25, Index=i
    Key : Lower : Value      : Upper : Fixed : Stale : Domain
      1 :  None :        0.0 :  None : False : False :  Reals
      2 :  None :        0.0 :  None : False : False :  Reals
      3 :  None :        0.0 :  None : False : False :  Reals
      4 :  None :        0.0 :  None : False : False :  Reals
      5 :  None :        0.0 :  None : False : False :  Reals
      6 :  None :        0.0 :  None : False : False :  Reals
      7 :  None :        0.0 :  None : False : False :  Reals
      8 :  None : 0.62405786 :  None : False : False :  Reals
      9 :  None :        0.0 :  None : False : False :  Reals
     10 :  None :  1.3184742 :  None : False : False :  Reals
     11 :  None :  0.2865239 :  None : False : False :  Reals
     12 :  None :        0.0 :  None : False : False :  Reals
     13 :  None :        0.0 :  None : False : False :  Reals
     14 :  None :        0.0 :  None : False : False :  Reals
     15 :  None : 0.14740946 :  None : False : Fa

##### Computing Average Performance

In [ ]:
N = 25
l = 1
d = False

res_adam_nt = []
while len(res_adam_nt) <= 30:
  model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
  solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

  start_time = time.time()
  result = solver.solve(model)
  solve_time = (time.time() - start_time)
  print(f"Solve Time in Min: {solve_time/60}")
  # check the status and termination condition of the result
  if result.solver.status == pyo.SolverStatus.ok and result.solver.termination_condition == pyo.TerminationCondition.optimal:
    # access the objective value
    obj_val = pyo.value(model.obj)
    print(f"Objective value: {obj_val}")
    # save the objective value to the res list
    res_adam_nt.append(obj_val)
  else:
    # the problem was not solved optimally
    print("The problem was not solved optimally")
    print(f"Termination condition: {result.solver.termination_condition}")

Solve Time in Min: 0.004818089803059896
Objective value: 19.2246359904
Solve Time in Min: 0.00600745677947998
Objective value: 15.048367578596999
Solve Time in Min: 0.014658749103546143
Objective value: 13.063466213914998
Solve Time in Min: 0.008191521962483723
Objective value: 14.013607359399998
Solve Time in Min: 0.006598182519276937
Objective value: 8.019243143130002
Solve Time in Min: 0.013962360223134358
Objective value: 16.109333301160003
Solve Time in Min: 0.011504451433817545
Objective value: 11.952254074540003
Solve Time in Min: 0.017780176798502603
Objective value: 12.839913472799998
Solve Time in Min: 0.01357202132542928
Objective value: 15.0380760568
Solve Time in Min: 0.015346844991048178
Objective value: 15.018618836960004
Solve Time in Min: 0.018854467074076335
Objective value: 15.000536617919998
Solve Time in Min: 0.03254065116246541
Objective value: 13.0328737619
Solve Time in Min: 0.011861129601796468
Objective value: 14.14101174433
Solve Time in Min: 0.01233679453531

In [ ]:
print(f"mean result = {np.mean(res_adam_nt)}, std result = {np.std(res_adam_nt)}")

mean result = 14.315542960536968, std result = 2.272821177993103


#### SGD

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#Setting Nurons number per hidden layers
n_neurons = 10
l = 1
n_size = l * [n_neurons]

# Create an MLPRegressor object
mlp = MLPRegressor(hidden_layer_sizes=n_size, activation='relu', max_iter=3000, solver='sgd', random_state=3)

# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the MLP to the data to compute the initial weights
mlp.fit(X_train, y_train)

#Getting weights and bias
weights = mlp.coefs_
bias = mlp.intercepts_

In [ ]:
mlp.score(X_test, y_test)

0.8843476598285834

In [ ]:
%%time
import pandas as pd
import time
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor

N = 25
l = 1
d = False

model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

start_time = time.time()
result = solver.solve(model)
solve_time = (time.time() - start_time)
print(f"Solve Time in Min: {solve_time/60}")
pyo.display(model.obj)

Solve Time in Min: 0.019126482804616294
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 12.02837063
CPU times: user 366 ms, sys: 31.8 ms, total: 398 ms
Wall time: 1.43 s


In [ ]:
model.y.pprint()

y : Size=25, Index=i
    Key : Lower : Value        : Upper : Fixed : Stale : Domain
      1 :  None :   0.97572721 :  None : False : False :  Reals
      2 :  None :  -0.10054815 :  None : False : False :  Reals
      3 :  None : -0.090160298 :  None : False : False :  Reals
      4 :  None :   0.92280911 :  None : False : False :  Reals
      5 :  None :    0.6899641 :  None : False : False :  Reals
      6 :  None : -0.088066597 :  None : False : False :  Reals
      7 :  None :    1.0080432 :  None : False : False :  Reals
      8 :  None :  -0.10721187 :  None : False : False :  Reals
      9 :  None :    1.0406199 :  None : False : False :  Reals
     10 :  None :   0.81606107 :  None : False : False :  Reals
     11 :  None :   0.86104295 :  None : False : False :  Reals
     12 :  None :    1.0249687 :  None : False : False :  Reals
     13 :  None :  -0.10134527 :  None : False : False :  Reals
     14 :  None :   0.91146699 :  None : False : False :  Reals
     15 :  None : -

In [ ]:
model.x.pprint()

x : Size=25, Index=i
    Key : Lower : Value      : Upper : Fixed : Stale : Domain
      1 :  None :        0.0 :  None : False : False :  Reals
      2 :  None :        0.0 :  None : False : False :  Reals
      3 :  None :        0.0 :  None : False : False :  Reals
      4 :  None :        0.0 :  None : False : False :  Reals
      5 :  None :  0.5354615 :  None : False : False :  Reals
      6 :  None :        0.0 :  None : False : False :  Reals
      7 :  None :        0.0 :  None : False : False :  Reals
      8 :  None :        0.0 :  None : False : False :  Reals
      9 :  None :        0.0 :  None : False : False :  Reals
     10 :  None :  1.9892994 :  None : False : False :  Reals
     11 :  None :  2.1958958 :  None : False : False :  Reals
     12 :  None :        0.0 :  None : False : False :  Reals
     13 :  None :        0.0 :  None : False : False :  Reals
     14 :  None :        0.0 :  None : False : False :  Reals
     15 :  None :        0.0 :  None : False : Fa

##### Computing Average Performance

In [ ]:
N = 25
l = 1
d = False

res_sgd = []
while len(res_sgd) <= 30:
  model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
  solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

  start_time = time.time()
  result = solver.solve(model)
  solve_time = (time.time() - start_time)
  print(f"Solve Time in Min: {solve_time/60}")
  # check the status and termination condition of the result
  if result.solver.status == pyo.SolverStatus.ok and result.solver.termination_condition == pyo.TerminationCondition.optimal:
    # access the objective value
    obj_val = pyo.value(model.obj)
    print(f"Objective value: {obj_val}")
    # save the objective value to the res list
    res_sgd.append(obj_val)
  else:
    # the problem was not solved optimally
    print("The problem was not solved optimally")
    print(f"Termination condition: {result.solver.termination_condition}")

Solve Time in Min: 0.01256546974182129
Objective value: 16.263639038
Solve Time in Min: 0.01078250010808309
Objective value: 14.159042697999999
Solve Time in Min: 0.03688872257868449
Objective value: 12.361037853999997
Solve Time in Min: 0.018603440125783285
Objective value: 6.912532199999998
Solve Time in Min: 0.010484580198923748
Objective value: 10.876032205000001
Solve Time in Min: 0.011595110098520914
Objective value: 9.827035408000004
Solve Time in Min: 0.03671725988388062
Objective value: 10.689633364999999
Solve Time in Min: 0.020042864481608073
Objective value: 11.282063087000001
Solve Time in Min: 0.04826375643412272
Objective value: 12.360424365000004
Solve Time in Min: 0.01717193126678467
Objective value: 12.738272539
Solve Time in Min: 0.008712267875671387
Objective value: 11.585489778999998
Solve Time in Min: 0.008346537748972574
Objective value: 14.6749114125
Solve Time in Min: 0.018980626265207926
Objective value: 14.358409618999996
Solve Time in Min: 0.0238327701886494

In [ ]:
print(f"mean result = {np.mean(res_sgd)}, std result = {np.std(res_sgd)}")

mean result = 11.882586438048389, std result = 2.797347774775704


#### lbfgs

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#Setting Nurons number per hidden layers
n_neurons = 10
l = 1
n_size = l * [n_neurons]

# Create an MLPRegressor object
mlp = MLPRegressor(hidden_layer_sizes=n_size, activation='relu', max_iter=1000, solver='lbfgs', random_state=3)

# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the MLP to the data to compute the initial weights
mlp.fit(X_train, y_train)

#Getting weights and bias
weights = mlp.coefs_
bias = mlp.intercepts_

In [ ]:
mlp.score(X_test, y_test)

0.9994086460179425

In [ ]:
%%time
import pandas as pd
import time
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor

N = 25
l = 1
d = False

model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

start_time = time.time()
result = solver.solve(model)
solve_time = (time.time() - start_time)
print(f"Solve Time in Min: {solve_time/60}")
pyo.display(model.obj)

Solve Time in Min: 0.14032047589619953
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 18.00683143682
CPU times: user 196 ms, sys: 92.7 ms, total: 289 ms
Wall time: 8.47 s


In [ ]:
model.y.pprint()

y : Size=25, Index=i
    Key : Lower : Value          : Upper : Fixed : Stale : Domain
      1 :  None :      0.9997866 :  None : False : False :  Reals
      2 :  None :      1.0004806 :  None : False : False :  Reals
      3 :  None :     0.99915703 :  None : False : False :  Reals
      4 :  None :      1.0001131 :  None : False : False :  Reals
      5 :  None :  -0.0005770967 :  None : False : False :  Reals
      6 :  None :      1.0004234 :  None : False : False :  Reals
      7 :  None :      1.0016292 :  None : False : False :  Reals
      8 :  None :     0.99805376 :  None : False : False :  Reals
      9 :  None :      1.0009689 :  None : False : False :  Reals
     10 :  None :     0.99810297 :  None : False : False :  Reals
     11 :  None :      1.0008599 :  None : False : False :  Reals
     12 :  None :  -0.0018988177 :  None : False : False :  Reals
     13 :  None :   0.0036441798 :  None : False : False :  Reals
     14 :  None : -0.00059483558 :  None : False : Fals

In [ ]:
model.x.pprint()

x : Size=25, Index=i
    Key : Lower : Value      : Upper : Fixed : Stale : Domain
      1 :  None :  0.9426016 :  None : False : False :  Reals
      2 :  None :        0.0 :  None : False : False :  Reals
      3 :  None :        0.0 :  None : False : False :  Reals
      4 :  None :        0.0 :  None : False : False :  Reals
      5 :  None :        0.0 :  None : False : False :  Reals
      6 :  None :        0.0 :  None : False : False :  Reals
      7 :  None :        0.0 :  None : False : False :  Reals
      8 :  None : 0.64994705 :  None : False : False :  Reals
      9 :  None :        0.0 :  None : False : False :  Reals
     10 :  None : 0.82865406 :  None : False : False :  Reals
     11 :  None :        0.0 :  None : False : False :  Reals
     12 :  None :        0.0 :  None : False : False :  Reals
     13 :  None :        0.0 :  None : False : False :  Reals
     14 :  None :        0.0 :  None : False : False :  Reals
     15 :  None :  1.0152167 :  None : False : Fa

##### Computing Average Performance

In [ ]:
N = 25
l = 1
d = False

res_lbfgs = []
i = 0
while len(res_lbfgs) <= 30:
  i += 1
  model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
  solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

  start_time = time.time()
  result = solver.solve(model)
  solve_time = (time.time() - start_time)
  print(f"Solve Time in Min: {solve_time/60}")
  # check the status and termination condition of the result
  if result.solver.status == pyo.SolverStatus.ok and result.solver.termination_condition == pyo.TerminationCondition.optimal:
    # access the objective value
    obj_val = pyo.value(model.obj)
    print(f"Objective value: {obj_val}")
    # save the objective value to the res list
    res_lbfgs.append(obj_val)
  else:
    # the problem was not solved optimally
    print("The problem was not solved optimally")
    print(f"Termination condition: {result.solver.termination_condition}")

Solve Time in Min: 0.05658361117045085
Objective value: 10.005280567399998
Solve Time in Min: 0.21903436581293742
Objective value: 11.26151397001
Solve Time in Min: 1.4104634086290995
Objective value: 14.009964679889999
Solve Time in Min: 0.05394422213236491
Objective value: 13.019884299910002
Solve Time in Min: 0.07650994857152303
Objective value: 16.03019397501
Solve Time in Min: 0.03466349045435588
Objective value: 17.00416778091
Solve Time in Min: 0.1586639086405436
Objective value: 15.01323016948
Solve Time in Min: 0.22970503966013592
Objective value: 16.00670005846
Solve Time in Min: 0.0491742213567098
Objective value: 18.005394090994
Solve Time in Min: 0.12391060590744019
Objective value: 13.022131783090002
Solve Time in Min: 0.012346049149831137
Objective value: 12.016733372703298
Solve Time in Min: 0.16289546489715576
Objective value: 16.00309384197
Solve Time in Min: 0.04700699249903361
Objective value: 13.018784059099998
Solve Time in Min: 0.1081394910812378
Objective value:

In [ ]:
print(f"mean result = {np.mean(res_lbfgs)}, std result = {np.std(res_lbfgs)}")

mean result = 14.795933048740153, std result = 2.2104669343393084


# Heuristic

In [ ]:
s_test_backup = s_testset.copy()

### Computing Difference

In [ ]:
s_testset['max_merit'] = 2.5
s_testset['min_merit'] = 0
s_testset['p_max'] = 0
s_testset['p_min'] = 0

row_i_max = s_testset.loc[:, ['SAT', 'GPA', 'max_merit']]
row_i_max.columns = ['SAT', 'GPA', 'merit']
s_testset.loc[:, 'p_max'] = mlp.predict(row_i_max)

row_i_min = s_testset.loc[:, ['SAT', 'GPA', 'min_merit']]
row_i_min.columns = ['SAT', 'GPA', 'merit']
s_testset.loc[:, 'p_min'] = mlp.predict(row_i_min)
s_testset['diff_p'] = s_testset.loc[:, 'p_max'] -  s_testset.loc[:, 'p_min']
sorted_s_test = s_testset.sort_values(['diff_p'], ascending=False)

### Allocating Budget

In [ ]:
budget = int(0.2*N)
exp = 0
for i in sorted_s_test.index:
  if budget - 2.5 >= 0:
    budget -= 2.5
    exp += sorted_s_test.loc[i, 'p_max']

In [ ]:
print(f"Heuristic Result = {exp}")

Heuristic Result = 2.004179780238963


# JANOS

In [ ]:
! pip install janos gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 50.8 MB/s eta 0:00:00


In [ ]:
from janos import *
import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import time
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

In [ ]:

"""
set the constant in the model
"""
scholarships = [0, 2.5]  # lower and upper bound if the scholarship
n_simulations = 30  # to have meaningful mean and standard deviation;
student_sizes = [25]  # we measure these predictions' RMSE
LAYERS = 1
nodes_per_layer = 10
"""
pretrained model
"""
# Assign X and y
X = trainset[["SAT", "GPA", "merit"]]
y = trainset.loc[:, "enroll"]

"""
Prepare the output file
"""
now = datetime.now()
date_time = now.strftime("%H-%M-%S-%Y%m%d")
filename = "20200501_neural_network_" + date_time + ".txt"
output = open(filename, "w")
output.write("PM\t\tstudent_size\t\tn_layers\t\titeration\t\tjanos_time\t\tgurobi_time\t\tobj_val\n")
output.close()

for student_size in student_sizes:
    n_applications = student_size
    BUDGET = int(0.2 * n_applications)
    hidden_layer_sizes = []
    for n_layers in range(LAYERS):

        hidden_layer_sizes.append(nodes_per_layer)

    mlp = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes, random_state=0)  ### TODO: how to link training and optimization!
    mlp.fit(X[["SAT", "GPA", "merit"]], y)

    res_janos = []
    for iter in range(n_simulations):
        random_sample = testset.sample(student_size)
        random_sample = random_sample.reset_index()

        m = JModel()

        # Define regular variables
        assign_scholarship = m.add_regular_variables([n_applications], "assign_scholarship")
        for app_index in range(n_applications):
            assign_scholarship[app_index].setContinuousDomain(lower_bound=scholarships[0],
                                                              upper_bound=scholarships[1])
            assign_scholarship[app_index].setObjectiveCoefficient(0)

        # Define predicted variables
        # First, we need to create structures of predictive models. In this case, we associate such a structure with an existing / pretrained logistic regression model.
        logistic_regression_model = OptimizationPredictiveModel(m, pretrained_model=mlp,
                                                                feature_names=["SAT", "GPA", "merit"])

        # Now, we could define the predicted decision variables and associate them with the predicted model structure.
        enroll_probabilities = m.add_predicted_variables([n_applications], "enroll_probs")
        for app_index in range(n_applications):
            enroll_probabilities[app_index].setObjectiveCoefficient(1)
            mapping_of_vars = {"merit": assign_scholarship[app_index],
                                "SAT": random_sample["SAT"][app_index],
                                "GPA": random_sample["GPA"][app_index]}
            enroll_probabilities[app_index].setPM(logistic_regression_model, mapping_of_vars)

        # Construct constraints
        # \sum_i x_i <= BUDGET
        scholarship_deployed = Expression()

        for app_index in range(n_applications):
            scholarship_deployed.add_term(assign_scholarship[app_index], 1)

        m.add_constraint(scholarship_deployed, "less_equal", BUDGET)
        #            m.add_gurobi_param_settings("MIPGap", 0.01)

        # solve the model
        #m.add_gurobi_param_settings('TimeLimit', 1800)
        m.add_gurobi_param_settings('DUALREDUCTIONS', 0)
        m.add_gurobi_param_settings('MIPGap', 0.001)
        m.add_gurobi_param_settings('Threads', 1)
        m.set_output_flag(0)
        m.solve()

        """
        write output
        borrowed from https://www.gurobi.com/documentation/8.1/examples/workforce1_py.html
        """
        status = m.gurobi_model.status

        if status == GRB.Status.UNBOUNDED:
            print('The model cannot be solved because it is unbounded')
            sys.exit(0)
        elif status == GRB.Status.OPTIMAL:
            output = open(filename, "a")
            output.write("NN\t\t" + str(student_size) + "\t\t" + str(LAYERS) + "\t\t" + str(iter) +
                          "\t\t" + str(m.get_time()) + "\t\t" + str(m.gurobi_model.runtime) +
                          "\t\t" + str(m.gurobi_model.objval) + "\n")
            output.close()
            res_janos.append(m.gurobi_model.objval)

        elif status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
            print('Optimization was stopped with status %d' % status)
        else:
            # if none of the above, then do IIS
            print('The model is infeasible; computing IIS')
            m.gurobi_model.computeIIS()
            m.gurobi_model.write("ip_model_inf.ilp")
            if m.gurobi_model.IISMinimal:
                print('IIS is minimal\n')
            else:
                print('IIS is not minimal\n')
            print('\nThe following constraint(s) cannot be satisfied:')
            for c in m.gurobi_model.getConstrs():
                if c.IISConstr:
                    print('%s' % c.constrName)


Restricted license - for non-production use only - expires 2025-11-24
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600
Set parameter Time

In [ ]:
print(f"mean result = {np.mean(res_janos)}, std result = {np.std(res_janos)}")

mean result = 14.109749737732816, std result = 2.253116405874849


In [ ]:
m.get_best_solution()

{'regular': [0.0,
  0.0,
  0.0,
  1.3,
  0.0,
  0.0,
  0.0,
  1.3,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.49,
  0.0,
  1.24,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.67,
  0.0,
  0.0],
 'predicted': [0.01,
  1.0,
  0.01,
  1.0,
  1.0,
  0.0,
  0.01,
  1.0,
  0.01,
  1.0,
  0.0,
  -0.0,
  0.0,
  0.93,
  1.0,
  0.99,
  1.0,
  0.99,
  0.01,
  1.0,
  0.0,
  0.01,
  0.95,
  1.0,
  0.01]}

# Statistical Tests

## Implemented Model and JANOS are the same

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(res_adam, res_janos,equal_var=False, alternative='two-sided')

TtestResult(statistic=1.2099739144228867, pvalue=0.231134799594279, df=58.75175552071136)

## Effect of Different MLP Solver

In [ ]:
from scipy import stats
stats.kruskal(res_adam, res_adam_nt, res_sgd, res_lbfgs)

KruskalResult(statistic=21.09255359001037, pvalue=0.0001007180695094934)

Excluding SGD that has lower test accuracy

In [ ]:
stats.kruskal(res_adam, res_adam_nt, res_lbfgs)

KruskalResult(statistic=0.6913897314411201, pvalue=0.7077284063599573)

## Tuning and Not Tuning Model

In [ ]:
ttest_ind(res_adam, res_adam_nt,equal_var=False, alternative='two-sided')

TtestResult(statistic=0.8800356329650662, pvalue=0.38237971608393984, df=59.51658223252582)

# Test on N=100

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#Setting Nurons number per hidden layers
n_neurons = 10
l = 1
n_size = l * [n_neurons]

# Create an MLPRegressor object
mlp = MLPRegressor(hidden_layer_sizes=n_size, activation='relu', max_iter=500, solver='adam', random_state=3, **dict_l_alpha[l])

# Separate the input features (X) and the target variable (y)
X = trainset.loc[:, ['SAT', 'GPA', 'merit']]
y = trainset.loc[:, 'enroll']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Fit the MLP to the data to compute the initial weights
mlp.fit(X_train, y_train)

#Getting weights and bias
weights = mlp.coefs_
bias = mlp.intercepts_

In [ ]:
mlp.score(X_test, y_test)

0.9612498114793685

In [ ]:
%%time
import pandas as pd
import time
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor

N = 100
l = 1
d = False

model, s_testset = Modeling(N, l, testset, weights, bias, X.shape[1], 10, d)
solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc', tee=True)

start_time = time.time()
result = solver.solve(model)
solve_time = (time.time() - start_time)
print(f"Solve Time in Min: {solve_time/60}")
pyo.display(model.obj)

Solve Time in Min: 15.146363667647044
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 59.03778642918
CPU times: user 5.46 s, sys: 630 ms, total: 6.08 s
Wall time: 15min 9s
